## Gerador de DataSet em CSV para o Exercício
Execute o Script abaixo para gerar o Dataset para ser utilizado no [exercicio](../exercicio.md)

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType
import os

# 1. Inicialização da SparkSession (garantir que está rodando)
spark = SparkSession.builder.appName("DesafioLogistica").getOrCreate()

# --- DADOS BRUTOS (Substitui o createDataFrame) ---

# CORREÇÃO: Usamos o caminho ABSOLUTO do contêiner para o volume 'data'
caminho_local_data = "data/"
nome_arquivo = "logistica_raw.csv"
caminho_completo = os.path.join(caminho_local_data, nome_arquivo)

csv_content = """ID_RASTREIO,DATA_ENVIO_RAW,CUSTO_FRETE_RAW,CIDADE_DESTINO,STATUS_ENTREGA
L9001,2024-05-10,15.50,Sao Paulo,ENTREGUE
L9002,2024/05/11,22,00,Rio,Em Trânsito
L9003,2024-05-12,100.00,Salvador,
L9004,2024-05-12,,BH,PENDENTE
L9004,2024-05-12,,BH,PENDENTE
"""
# A escrita deve ser feita no caminho_completo
with open(caminho_completo, "w") as f:
    f.write(csv_content)
print(f"Arquivo de Logística Bruto gerado em: {caminho_completo}")

# --- PONTO DE PARTIDA DO ALUNO: LEITURA ---

# Defina o Schema (TODAS COMO StringType)
schema_logistica = StructType([
    StructField("ID_RASTREIO", StringType(), nullable=False),
    StructField("DATA_ENVIO_RAW", StringType(), nullable=True),
    StructField("CUSTO_FRETE_RAW", StringType(), nullable=True),
    StructField("CIDADE_DESTINO", StringType(), nullable=True),
    StructField("STATUS_ENTREGA", StringType(), nullable=True)
])

# Carregar o DataFrame para teste
# A leitura deve usar o mesmo caminho_completo
df_logistica_raw = (
    spark.read
    .csv(
        caminho_completo,
        header=True,
        schema=schema_logistica,
        sep=","
    )
)

print("\n--- INGESTÃO BEM-SUCEDIDA (5 linhas brutas) ---")
df_logistica_raw.show()
df_logistica_raw.printSchema()


Arquivo de Logística Bruto gerado em: data/logistica_raw.csv

--- INGESTÃO BEM-SUCEDIDA (5 linhas brutas) ---
+-----------+--------------+---------------+--------------+--------------+
|ID_RASTREIO|DATA_ENVIO_RAW|CUSTO_FRETE_RAW|CIDADE_DESTINO|STATUS_ENTREGA|
+-----------+--------------+---------------+--------------+--------------+
|      L9001|    2024-05-10|          15.50|     Sao Paulo|      ENTREGUE|
|      L9002|    2024/05/11|             22|            00|           Rio|
|      L9003|    2024-05-12|         100.00|      Salvador|          NULL|
|      L9004|    2024-05-12|           NULL|            BH|      PENDENTE|
|      L9004|    2024-05-12|           NULL|            BH|      PENDENTE|
+-----------+--------------+---------------+--------------+--------------+

root
 |-- ID_RASTREIO: string (nullable = true)
 |-- DATA_ENVIO_RAW: string (nullable = true)
 |-- CUSTO_FRETE_RAW: string (nullable = true)
 |-- CIDADE_DESTINO: string (nullable = true)
 |-- STATUS_ENTREGA: string

In [2]:
# Pré-requisito: Defina o StructType que trate todas as colunas como StringType.
schema_logistico = StructType([
    StructField("ID_RASTREIO", StringType(), nullable=False),
    StructField("DATA_ENVIO_RAW", StringType(), nullable=True), # O tipo original é String, pois o formato é inconsistente ('2024-03-20' vs '2024/03/20')
    StructField("CUSTO_FRETE_RAW", StringType(), nullable=True),  # O tipo original é String, pois pode vir com vírgula (250,99) ou nulo
    StructField("CIDADE_DESTINO", StringType(), nullable=True),
    StructField("STATUS_ENTREGA", StringType(), nullable=True)
])

df_logistico_raw = (
    spark.read
    .csv(
        caminho_completo,
        header=True,
        schema=schema_logistico,
        sep=",",
        # O PySpark lê a primeira linha como cabeçalho
        # e garante que os dados sigam o schema definido (Data Quality!)
    )
)

print("\n--- Schema Original (Raw) ---")
df_logistico_raw.printSchema()

print("\n--- Primeiros Registros ---")
df_logistico_raw.show(truncate=False)


--- Schema Original (Raw) ---
root
 |-- ID_RASTREIO: string (nullable = true)
 |-- DATA_ENVIO_RAW: string (nullable = true)
 |-- CUSTO_FRETE_RAW: string (nullable = true)
 |-- CIDADE_DESTINO: string (nullable = true)
 |-- STATUS_ENTREGA: string (nullable = true)


--- Primeiros Registros ---
+-----------+--------------+---------------+--------------+--------------+
|ID_RASTREIO|DATA_ENVIO_RAW|CUSTO_FRETE_RAW|CIDADE_DESTINO|STATUS_ENTREGA|
+-----------+--------------+---------------+--------------+--------------+
|L9001      |2024-05-10    |15.50          |Sao Paulo     |ENTREGUE      |
|L9002      |2024/05/11    |22             |00            |Rio           |
|L9003      |2024-05-12    |100.00         |Salvador      |NULL          |
|L9004      |2024-05-12    |NULL           |BH            |PENDENTE      |
|L9004      |2024-05-12    |NULL           |BH            |PENDENTE      |
+-----------+--------------+---------------+--------------+--------------+



In [3]:
# trate nulos na coluna STATUS_ENTREGA: Se for nulo, preencha com a string "DESCONHECIDO".
df_limpeza = df_logistico_raw.withColumn(
    "STATUS_ENTREGA",
    F.coalesce(F.col("STATUS_ENTREGA"), F.lit("DESCONHECIDO")) # Coalesce pega o primeiro valor não nulo
)

# Trate nulos na coluna CUSTO_FRETE_RAW: Se for nulo, preencha com a string "0.00"
df_limpeza = df_limpeza.na.fill(value="0.0", subset=['CUSTO_FRETE_RAW'])

print("\n--- 1. Limpeza (Cleaning): Nulos Tratados ---")
df_limpeza.show(truncate=False)


--- 1. Limpeza (Cleaning): Nulos Tratados ---
+-----------+--------------+---------------+--------------+--------------+
|ID_RASTREIO|DATA_ENVIO_RAW|CUSTO_FRETE_RAW|CIDADE_DESTINO|STATUS_ENTREGA|
+-----------+--------------+---------------+--------------+--------------+
|L9001      |2024-05-10    |15.50          |Sao Paulo     |ENTREGUE      |
|L9002      |2024/05/11    |22             |00            |Rio           |
|L9003      |2024-05-12    |100.00         |Salvador      |DESCONHECIDO  |
|L9004      |2024-05-12    |0.0            |BH            |PENDENTE      |
|L9004      |2024-05-12    |0.0            |BH            |PENDENTE      |
+-----------+--------------+---------------+--------------+--------------+



In [6]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType, DateType

# Crie uma coluna DATA_ENVIO convertendo DATA_ENVIO_RAW para o tipo Date.
# Crie uma coluna CUSTO_FRETE convertendo CUSTO_FRETE_RAW para o tipo Double (remova vírgulas antes do cast).
# Converta a coluna STATUS_ENTREGA para CAIXA ALTA.
df_transformacao = (
    df_limpeza
    .withColumn("STATUS_ENTREGA", F.upper(F.col("STATUS_ENTREGA")))
    .withColumn(
        "CUSTO_FRETE",
        F.regexp_replace(F.col("CUSTO_FRETE_RAW"), ",", ".").cast(DoubleType())
    )

    .withColumn(
        "DATA_ENVIO",
        F.to_date(F.col("DATA_ENVIO_RAW"), "yyyy-MM-dd") # Tenta formato padrão, o PySpark pode inferir variações simples
    )
    .withColumn("DATA_ENVIO", F.coalesce(F.col("DATA_ENVIO"), F.to_date(F.col("DATA_ENVIO_RAW"), "yyyy/MM/dd"))) # Tenta o formato com barra, caso o anterior falhe
    .drop("DATA_ENVIO_RAW", "CUSTO_FRETE_RAW") # Remove as colunas RAW
)

print("\n--- 2. Transformação: Padronização e Casting ---")
df_transformacao.printSchema()
df_transformacao.show(truncate=False)


--- 2. Transformação: Padronização e Casting ---
root
 |-- ID_RASTREIO: string (nullable = true)
 |-- CIDADE_DESTINO: string (nullable = true)
 |-- STATUS_ENTREGA: string (nullable = false)
 |-- CUSTO_FRETE: double (nullable = true)
 |-- DATA_ENVIO: date (nullable = true)

+-----------+--------------+--------------+-----------+----------+
|ID_RASTREIO|CIDADE_DESTINO|STATUS_ENTREGA|CUSTO_FRETE|DATA_ENVIO|
+-----------+--------------+--------------+-----------+----------+
|L9001      |Sao Paulo     |ENTREGUE      |15.5       |2024-05-10|
|L9002      |00            |RIO           |22.0       |2024-05-11|
|L9003      |Salvador      |DESCONHECIDO  |100.0      |2024-05-12|
|L9004      |BH            |PENDENTE      |0.0        |2024-05-12|
|L9004      |BH            |PENDENTE      |0.0        |2024-05-12|
+-----------+--------------+--------------+-----------+----------+



In [7]:
# Crie uma coluna FLAG_CARO: Atribua True se CUSTO_FRETE for maior ou igual a 50.00, senão False.
# Ação: Criar uma nova coluna indicando se a venda é considerada de "Alto Valor" (> 200)
df_enriquecimento = df_transformacao.withColumn(
    "FLAG_CARO",
    F.when(F.col("CUSTO_FRETE") >= 50, F.lit(True)).otherwise(F.lit(False))
)

print("\n--- 3. Enriquecimento: Nova Feature Adicionada ---")
df_enriquecimento.show(truncate=False)


--- 3. Enriquecimento: Nova Feature Adicionada ---
+-----------+--------------+--------------+-----------+----------+---------+
|ID_RASTREIO|CIDADE_DESTINO|STATUS_ENTREGA|CUSTO_FRETE|DATA_ENVIO|FLAG_CARO|
+-----------+--------------+--------------+-----------+----------+---------+
|L9001      |Sao Paulo     |ENTREGUE      |15.5       |2024-05-10|false    |
|L9002      |00            |RIO           |22.0       |2024-05-11|false    |
|L9003      |Salvador      |DESCONHECIDO  |100.0      |2024-05-12|true     |
|L9004      |BH            |PENDENTE      |0.0        |2024-05-12|false    |
|L9004      |BH            |PENDENTE      |0.0        |2024-05-12|false    |
+-----------+--------------+--------------+-----------+----------+---------+



In [10]:
# Use ID_RASTREIO como chave e remova as duplicatas. O resultado final deve ter 4 linhas.
colunas_chave = ["ID_RASTREIO"]

# Ação: Remover duplicatas estritas
df_curated = df_enriquecimento.dropDuplicates(subset=colunas_chave)

print(f"\n--- 5. Desduplicação: Registros Únicos ({df_enriquecimento.count()} vs {df_curated.count()}) ---")
df_curated.show(truncate=False)
df_curated.printSchema()


--- 5. Desduplicação: Registros Únicos (5 vs 4) ---
+-----------+--------------+--------------+-----------+----------+---------+
|ID_RASTREIO|CIDADE_DESTINO|STATUS_ENTREGA|CUSTO_FRETE|DATA_ENVIO|FLAG_CARO|
+-----------+--------------+--------------+-----------+----------+---------+
|L9001      |Sao Paulo     |ENTREGUE      |15.5       |2024-05-10|false    |
|L9002      |00            |RIO           |22.0       |2024-05-11|false    |
|L9003      |Salvador      |DESCONHECIDO  |100.0      |2024-05-12|true     |
|L9004      |BH            |PENDENTE      |0.0        |2024-05-12|false    |
+-----------+--------------+--------------+-----------+----------+---------+

root
 |-- ID_RASTREIO: string (nullable = true)
 |-- CIDADE_DESTINO: string (nullable = true)
 |-- STATUS_ENTREGA: string (nullable = false)
 |-- CUSTO_FRETE: double (nullable = true)
 |-- DATA_ENVIO: date (nullable = true)
 |-- FLAG_CARO: boolean (nullable = false)



In [11]:
# Salve o DataFrame em: ./data/curated/logistica_parquet.
# Use o modo overwrite.
# Particione o dataset fisicamente pela coluna CIDADE_DESTINO.
# Definindo o caminho de saída (simulação de um Data Lake)
caminho_parquet = "data/curated/logistica_parquet"

# Ação: Salvar o DataFrame tratado (df_curated) em Parquet
(
    df_curated.write
    .mode("overwrite") # Sobrescreve se o diretório já existir
    .partitionBy("CIDADE_DESTINO") # Particionamento físico: otimiza queries que filtram por status
    .option("compression", "snappy") # Snappy é o padrão (bom trade-off)
    .parquet(caminho_parquet)
)

print(f"\n--- 1. Escrita em Parquet concluída ---")
print(f"Dados salvos e particionados em: {caminho_parquet}")


--- 1. Escrita em Parquet concluída ---
Dados salvos e particionados em: data/curated/logistica_parquet


In [12]:
# Crie uma variável df_salvador. Leia o arquivo Parquet filtrando apenas as entregas com CIDADE_DESTINO igual a "Salvador".
df_salvador = spark.read.parquet(caminho_parquet).filter(F.col("CIDADE_DESTINO") == "Salvador")

print("\n--- 3.1. Predicate Pushdown: Filtro no Disco ---")
print(f"Total de Registros lidos: {df_salvador.count()}")
df_salvador.show()


--- 3.1. Predicate Pushdown: Filtro no Disco ---
Total de Registros lidos: 1
+-----------+--------------+-----------+----------+---------+--------------+
|ID_RASTREIO|STATUS_ENTREGA|CUSTO_FRETE|DATA_ENVIO|FLAG_CARO|CIDADE_DESTINO|
+-----------+--------------+-----------+----------+---------+--------------+
|      L9003|  DESCONHECIDO|      100.0|2024-05-12|     true|      Salvador|
+-----------+--------------+-----------+----------+---------+--------------+



In [13]:
# Crie uma variável df_leve. Leia o arquivo Parquet selecionando (projetando) apenas as colunas ID_RASTREIO e FLAG_CARO.
df_projecao = spark.read.parquet(caminho_parquet).select("ID_RASTREIO", "FLAG_CARO")

print("\n--- 3.2. SerDe: Deserialização Seletiva (Projeção) ---")
print(f"Colunas carregadas: {df_projecao.columns}")
df_projecao.printSchema()
df_projecao.show()


--- 3.2. SerDe: Deserialização Seletiva (Projeção) ---
Colunas carregadas: ['ID_RASTREIO', 'FLAG_CARO']
root
 |-- ID_RASTREIO: string (nullable = true)
 |-- FLAG_CARO: boolean (nullable = true)

+-----------+---------+
|ID_RASTREIO|FLAG_CARO|
+-----------+---------+
|      L9003|     true|
|      L9001|    false|
|      L9004|    false|
|      L9002|    false|
+-----------+---------+

